In [2]:
from pydrake.all import ModelVisualizer, Simulator, StartMeshcat
from manipulation import ConfigureParser, running_as_notebook
from manipulation.station import LoadScenario, MakeHardwareStation

from typing import Callable

import numpy as np
from pydrake.all import (
    BasicVector,
    Box,
    ConstantVectorSource,
    Context,
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    MathematicalProgram,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    SnoptSolver,
    PiecewisePolynomial,
    TrajectorySource,
    ge,
    le,
    Solve,
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_differential_ik import TestDifferentialIK
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.utils import RenderDiagram
from pydrake.multibody.tree import JacobianWrtVariable
import numpy as np

In [3]:
class DifferentialIKSystem(LeafSystem):
    def __init__(self, plant: MultibodyPlant, diffik_fun):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()
        self._diffik_fun = diffik_fun

        self.DeclareVectorInputPort("spatial_velocity", 6)
        self.DeclareVectorInputPort("iiwa_position_measured", 7)
        self.DeclareVectorInputPort("iiwa_velocity_estimated", 7)
        self.DeclareVectorOutputPort("iiwa_velocity_command", 7, self.CalcOutput)

    def CalcOutput(self, context, output: BasicVector):
        V_G_desired = self.get_input_port(0).Eval(context)
        q_now = self.get_input_port(1).Eval(context)
        v_now = self.get_input_port(2).Eval(context)

        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kQDot,
            self._G,
            [0, 0, 0],
            self._W,
            self._W
        )[:, 0:7]

        X_now = self._plant.CalcRelativeTransform(self._plant_context, self._W, self._G)
        p_now = X_now.translation()
        v = self._diffik_fun(J_G, V_G_desired, q_now, v_now, p_now)
        output.SetFromVector(v)

In [4]:
def DiffIKQP(J_G, V_G_desired, q_now, v_now, p_now):
    from pydrake.solvers import MathematicalProgram, SnoptSolver
    import numpy as np

    prog = MathematicalProgram()
    v = prog.NewContinuousVariables(7, "v")
    v_max = 3.0

    # Minimize (J*v - V_desired)^2
    prog.AddQuadraticCost((J_G @ v - V_G_desired) @ (J_G @ v - V_G_desired))

    # Bound each joint velocity
    prog.AddBoundingBoxConstraint(-v_max*np.ones(7), v_max*np.ones(7), v)

    solver = SnoptSolver()
    result = solver.Solve(prog)
    if not result.is_success():
        raise ValueError("Could not find optimal solution.")
    return result.GetSolution(v)

In [5]:
scenario_data = """
        directives:
        - add_model:
            name: iiwa
            file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
            default_joint_positions:
                iiwa_joint_1: [-1.57]
                iiwa_joint_2: [0.1]
                iiwa_joint_3: [0]
                iiwa_joint_4: [-1.2]
                iiwa_joint_5: [0]
                iiwa_joint_6: [1.6]
                iiwa_joint_7: [0]

        - add_model:
            name: wsg
            file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
            
        - add_weld:
            parent: world
            child:  iiwa::iiwa_link_0

        - add_weld:
            parent: iiwa::iiwa_link_7
            child: wsg::body
            X_PC:
                translation: [0, 0, 0.09]
                rotation: !Rpy { deg: [90, 0, 90]}

        - add_model:
            name: hoop_model
            file: file:///workspaces/6.4210-final-project/sdfs/basketball_hoop.sdf

        - add_weld:
            parent: world
            child: hoop_model::base_link_hoop  
            X_PC:
                translation: [5, 0, 3.048]
                rotation: !Rpy { deg: [0, 0, 180] }

        - add_model:
            name: ball
            file: package://drake_models/manipulation_station/sphere.sdf

        - add_weld:
            parent: world
            child: ball::base_link
            X_PC:
                translation: [0.5, 0, 0]
                rotation: !Rpy { deg: [0, 0, 0] }

        model_drivers:
            iiwa: !IiwaDriver
                hand_model_name: wsg
            wsg: !SchunkWsgDriver {}
    """


In [6]:
def BuildAndSimulate(diffik_fun, duration=7.0):
    builder = DiagramBuilder()
    scenario = LoadScenario(data=scenario_data)
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat=meshcat))
    
    plant = station.GetSubsystemByName("plant")
    controller = builder.AddSystem(DifferentialIKSystem(plant, diffik_fun))
    integrator = builder.AddSystem(Integrator(7))

    # ---- Pick-and-throw velocity trajectory ----
    times = [0.0, 1.0, 2.0, 4.0, 5.0, duration]
    velocities = [
        [0, 0, 0, 0.1, 0, 0],   # move above ball
        [0, 0, 0, -0.1, 0, 0],  # lower to grasp
        [0, 0, 0, 0, 0, 0],     # grasp
        [0, 0, 0, 0.2, 0, 0],   # lift
        [0, 0, 0, 0.5, 0, 0],   # throw
        [0, 0, 0, 0, 0, 0]      # stop
    ]
    traj = PiecewisePolynomial.FirstOrderHold(times, np.array(velocities).T)
    desired_vel = builder.AddSystem(TrajectorySource(traj))

    # ---- Build connections first ----
    builder.Connect(controller.get_output_port(), integrator.get_input_port())
    builder.Connect(integrator.get_output_port(), station.GetInputPort("iiwa.position"))
    builder.Connect(station.GetOutputPort("iiwa.position_measured"), controller.get_input_port(1))
    builder.Connect(station.GetOutputPort("iiwa.velocity_estimated"), controller.get_input_port(2))
    builder.Connect(desired_vel.get_output_port(), controller.get_input_port(0))

    # ---- Add gripper trajectory BEFORE building the diagram ----
    gripper_times = [0.0, 2.0, 5.0, duration]  # key times
    gripper_positions = [[0.1, 0.0, 0.1, 0.1]]  # open -> close -> open
    gripper_traj = PiecewisePolynomial.FirstOrderHold(
        gripper_times, np.array([0.1, 0.0, 0.1, 0.1]).reshape(1, -1)
    )
    gripper_source = builder.AddSystem(TrajectorySource(gripper_traj))
    builder.Connect(gripper_source.get_output_port(), station.GetInputPort("wsg.position"))

    # ---- Now build the diagram and create simulator ----
    diagram = builder.Build()
    simulator = Simulator(diagram)

    # Get contexts
    context = simulator.get_mutable_context()
    station_context = station.GetMyContextFromRoot(context)

    # Fix the WSG gripper input (already done via TrajectorySource)
    # Fix iiwa torque input to zero
    station.GetInputPort("iiwa.torque").FixValue(station_context, np.zeros(7))

    # Now advance
    simulator.AdvanceTo(duration)
    meshcat.PublishRecording()

In [7]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [8]:
simulator = BuildAndSimulate(
    DiffIKQP, duration=7.0 if running_as_notebook else 0.1
)